In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import json
from datetime import datetime
import time

In [2]:
ts_start = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
print('START TS: ',ts_start)

START TS:  21/01/2023 16:22:37


In [3]:
base_url = 'https://www.parquedosleiloes.com.br/leiloes?type_id=1&searchMode=normal' # type_id=1 Identifies that the auctions are from vehicles
page = requests.get(base_url)
page_soup = BeautifulSoup(page.content, 'html.parser')

In [4]:
## Get max auction pages
page_itens = page_soup.find_all("li", {"class": "page-item"})
max_auction_page = int(page_itens[-2].findChild("a").text)

In [5]:
SLEEP_TIME = 0.2
auctions_list = []
for i in range(0,max_auction_page+1): ## Search for every search pagination
    print('-'*15+'NEW PAGE'+'-'*15)
    print('Search Auction Page: ',i, 'out of ', max_auction_page)
    
    try:
        page = requests.get(base_url+f'&page={i}')
    except:
        try:
            page = requests.get(base_url+f'&page={i}')
        except:
            continue
    page_soup = BeautifulSoup(page.content, 'html.parser')
    auctions = page_soup.find_all("div", {"class": "card"})
    for a in range(0,len(auctions)): ## Get all auctions in this page
        auction = auctions[a]
        try:
            auction_name = auction.find_all('span',{'class':'name'})[0].text
        except:
            auction_name = ''
        try:
            auction_category = auction.find_all('small',{'class':'category'})[0].text
        except:
            auction_category = ''
        try:
            auction_date = auction.find_all('small',{'class':'date'})[0].text
        except:
            auction_date = ''
        try:
            auction_status = auction.find_all('span',{'class':'badge'})[0].text
        except:
            auction_status = ''
        try:
            auction_url = auction.find_all('a',{'class':'btn-secondary'})[0]['href']
            auction_number = auction_url.split('/')[-2]
        except:
            auction_url = ''
            auction_number = ''
        try:
            auction_edital_url = auction.find_all('a',{'class':'btn-primary'})[0]['href']
        except:
            auction_edital_url = ''
        if auction_name != 'BANCOS E FINANCEIRAS' or auction_number == '':
            continue
        
        ## Getting html page for each auction
        print('Getting auction: ',auction_number, ' URL: ', auction_url)
        try:
            auction_page = requests.get(auction_url)
        except:
            try:
                auction_page = requests.get(auction_url)
            except:
                continue
        auction_soup = BeautifulSoup(auction_page.content, 'html.parser')
        ## Get max auction itens
        page_itens = auction_soup.find_all("li", {"class": "page-item"})
        try:
            max_auction_page_item = int(page_itens[-2].findChild("a").text)
        except:
            max_auction_page_item = 0
        auction_itens_list = []
        for pi in range(1,max_auction_page_item+1): ## Search for every item pagination
            auction_page_url = auction_url+f'?page={pi}'
            print('\tSearching for auction item page: ', pi )
            
            try:
                auction_page = requests.get(auction_page_url)
            except:
                try:
                    auction_page = requests.get(auction_page_url)
                except:
                    continue
            auction_soup = BeautifulSoup(auction_page.content, 'html.parser')
            try:
                auction_itens = auction_soup.find_all("div", {"class": "auction-lot-card"})
            except:
                auction_itens = []
            
            for ai in range(0,len(auction_itens)): ## Get all itens in this page
                time.sleep(SLEEP_TIME) ## Sleep time to not exeed request times
                auction_item = auction_itens[ai]
                try:
                    auction_item_url = auction_item.find_all('a',{'class':'btn-secondary'})[0]['href']
                except:
                    continue
                print('\t\tSearching fo item URL: ',auction_item_url)
                if len(auction_item.find_all('span',{'class':'canceled'})) > 0:
                    print('\t\t\tAuction item canceled')
                    continue
                try:
                    auction_item_lote = auction_item.find_all('span',{'class':'badge-primary'})[0].find_all('b')[0].text
                except:
                    auction_item_lote = ''
                try:
                    auction_item_type = auction_item.find_all('span',{'class':'badge-type'})[0].text
                except:
                    auction_item_type = ''
                try:
                    auction_item_name = auction_item.find_all('span',{'class':'name'})[0].text
                except:
                    auction_item_name = ''
                
                try:
                    auction_item_page = requests.get(auction_item_url)
                except:
                    try:
                        auction_item_page = requests.get(auction_item_url)
                    except:
                        continue
                auction_item_soup = BeautifulSoup(auction_item_page.content, 'html.parser')

                ## Extract data from auction item
                try:
                    auction_item_min_bid_value = auction_item_soup.find_all('div',{'data-name':'min_bid_value'})[0]['data-value']
                except:
                    auction_item_min_bid_value = ''
                try:
                    auction_item_increment_value = auction_item_soup.find_all('div',{'data-name':'increment_value'})[0]['data-value']
                except:
                    auction_item_increment_value = ''
                try:
                    auction_item_highest_bid = auction_item_soup.find_all('div',{'data-name':'highest_bid'})[0]['data-value']
                except:
                    auction_item_highest_bid = ''
                try:
                    auction_item_date_text = auction_item_soup.find_all('div',{'class':'text-inner'})[0].find_all('p')[0].text
                except:
                    auction_item_date_text = ''
                try:
                    auction_item_description = auction_item_soup.find_all('table',{'class':'mb-10'})[0].find_all('td')[-1].find_all('p')[-1].text.replace('\xa0',' ')
                except:
                    auction_item_description = ''
                try:
                    auction_item_model = auction_item_soup.find_all('table',{'class':'mb-0'})[0].find_all('tr')[0].find_all('td')[-1].text
                except:
                    auction_item_model = ''
                try:
                    auction_item_brand = auction_item_soup.find_all('table',{'class':'mb-0'})[0].find_all('tr')[1].find_all('td')[-1].text
                except:
                    auction_item_brand = ''
                try:
                    auction_item_model_year = auction_item_soup.find_all('table',{'class':'mb-0'})[0].find_all('tr')[2].find_all('td')[-1].text
                except:
                    auction_item_model_year = ''
                try:
                    auction_item_market_price = auction_item_soup.find_all('table',{'class':'mb-0'})[0].find_all('tr')[3].find_all('td')[-1].text
                except:
                    auction_item_market_price = ''
                try:
                    auction_item_color = auction_item_soup.find_all('table',{'class':'mb-0'})[1].find_all('tr')[0].find_all('td')[-1].text
                except:
                    auction_item_color = ''
                try:
                    auction_item_gas_type = auction_item_soup.find_all('table',{'class':'mb-0'})[1].find_all('tr')[1].find_all('td')[-1].text
                except:
                    auction_item_gas_type = ''
                try:
                    auction_item_km = auction_item_soup.find_all('table',{'class':'mb-0'})[1].find_all('tr')[2].find_all('td')[-1].text
                except:
                    auction_item_km = ''
                try:
                    auction_item_refurbished = auction_item_soup.find_all('table',{'class':'mb-0'})[1].find_all('tr')[3].find_all('td')[-1].text
                except:
                    auction_item_refurbished = ''
                try:
                    auction_item_optionals = [opt.text for opt in auction_item_soup.find_all('div',{'class':'optionals'})[0].find_all('li')]
                except:
                    auction_item_optionals = []
                try:
                    auction_item_bids = auction_item_soup.find_all('table',{'class':'last-bids-table'})[0].find_all('tr')
                except:
                    auction_item_bids = []
                auction_bids = []
                auction_bid_dict = {}
                for bi in range(len(auction_item_bids)):
                    auction_item_bid = auction_item_bids[bi]
                    auction_item_bid_value = auction_item_bid['data-value']
                    try:
                        auction_item_bid_type = auction_item_bid.find_all('td')[0].find_all('div')[0]['title']
                    except:
                        auction_item_bid_type = ''
                    try:
                        auction_item_bid_datetime = auction_item_bid.find_all('td')[1].text
                    except:
                        auction_item_bid_datetime = ''
                    try:
                        auction_item_bid_user = auction_item_bid.find_all('td')[2].text.split(' ')[1]
                    except:
                        auction_item_bid_user = ''
                    auction_bid_dict = {
                                        'auction_item_bid_value': auction_item_bid_value,
                                        'auction_item_bid_type': auction_item_bid_type ,
                                        'auction_item_bid_datetime': auction_item_bid_datetime,
                                        'auction_item_bid_user': auction_item_bid_user
                                        }
                    auction_bids.append(auction_bid_dict)
                    
                auction_item_dict = {
                    'auction_item_lote':auction_item_lote,
                    'auction_item_type':auction_item_type,
                    'auction_item_name':auction_item_name,
                    'auction_item_url':auction_item_url,
                    'auction_item_min_bid_value':auction_item_min_bid_value,
                    'auction_item_increment_value':auction_item_increment_value,
                    'auction_item_highest_bid':auction_item_highest_bid,
                    'auction_item_date_text':auction_item_date_text,
                    'auction_item_description':auction_item_description,
                    'auction_item_model':auction_item_model,
                    'auction_item_brand':auction_item_brand,
                    'auction_item_model_year':auction_item_model_year,
                    'auction_item_market_price':auction_item_market_price,
                    'auction_item_color':auction_item_color,
                    'auction_item_gas_type':auction_item_gas_type,
                    'auction_item_km':auction_item_km,
                    'auction_item_refurbished':auction_item_refurbished,
                    'auction_item_optionals':auction_item_optionals,
                    'auction_item_bids':auction_bids
                }
                auction_itens_list.append(auction_item_dict)
        auctions_list.append(
            {
                'auction_number':auction_number,
                'auction_name':auction_name,
                'auction_category':auction_category,
                'auction_date':auction_date,
                'auction_status':auction_status,
                'auction_url':auction_url,
                'auction_edital_url':auction_edital_url,
                'auction_itens':auction_itens_list
            }
        )


---------------NEW PAGE---------------
Search Auction Page:  0 out of  23
Getting auction:  1081  URL:  https://www.parquedosleiloes.com.br/leilao/1081/detalhes
	Searching for auction item page:  1
		Searching fo item URL:  https://www.parquedosleiloes.com.br/leilao/1081/lote/52961
		Searching fo item URL:  https://www.parquedosleiloes.com.br/leilao/1081/lote/52962
		Searching fo item URL:  https://www.parquedosleiloes.com.br/leilao/1081/lote/52963
		Searching fo item URL:  https://www.parquedosleiloes.com.br/leilao/1081/lote/52964
		Searching fo item URL:  https://www.parquedosleiloes.com.br/leilao/1081/lote/52965
		Searching fo item URL:  https://www.parquedosleiloes.com.br/leilao/1081/lote/52966
		Searching fo item URL:  https://www.parquedosleiloes.com.br/leilao/1081/lote/52967
		Searching fo item URL:  https://www.parquedosleiloes.com.br/leilao/1081/lote/52968
		Searching fo item URL:  https://www.parquedosleiloes.com.br/leilao/1081/lote/52969
		Searching fo item URL:  https://www

In [6]:
ts_now = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
print('FINISH TS: ',ts_now)

FINISH TS:  21/01/2023 20:08:24


In [7]:

auctions = {
    'execution_ts': ts_now,
    'auctions': auctions_list
}

In [9]:
with open(f"./data/raw/auctions_{ts_now.replace('/','_').replace(':','_').replace(' ','_')}.json", 'w', encoding='utf-8') as f:
    json.dump(auctions, f)

In [22]:
#del auctions_list[-1]